<a href="https://colab.research.google.com/github/blackwingedkite/TMBA_100strategies_challenge/blob/master/chunking_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import re
import os

# 在一般的情況下切割純文字
def split_text(text, chunk_size):
    chunks = []
    current_chunk = ""
    for char in text:
        current_chunk += char
        if len(current_chunk) >= chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = ""
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def merge_chunk_content_new(txt_pages_dict, chunk_size=500):
    chunks = []
    text = "".join([page["text"] for page in txt_pages_dict])

    patterns = [
            (r'<table.*?>.*?</table>', 'table'),
            (r'image \d+:.*?end of image \d+', 'image'),
            # (r'<(div|p|h[1-6]).*?>.*?</\1>', 'html'),
            (r'ocr table \d+:.*?end of ocr table \d+:', 'ocr_table'),
            (r'single table \d+:.*?end of single table \d+', 'single_table'),
        ]

    # 在文本中找到所有匹配的模式
    matches = []
    for pattern, type in patterns:
        for match in re.finditer(pattern, text, re.DOTALL):
            matches.append((match.start(), match.end(), type, match.group()))
    matches.sort(key=lambda x: x[0])
    start_checkpoint = []
    end_checkpoint = []
    for i in matches:
        start_checkpoint.append(i[0])
        end_checkpoint.append(i[1])
        print(i)
        #(2725, 2924, 'image', 'image 0.........)
        #(4258, 5027, 'table', '<table>  ...)
        #(8941, 9900, 'single_table', 'single table 0...)
        #(8964, 9874, 'table', '<table ............)
    chunk_start = 0
    chunk_end = 0
    match_item = 0
    while True:
        if match_item < len(matches):
            if chunk_start + chunk_size >= len(text):
                print(str(chunk_start)+ " "+str(len(text)) )
                print("SHOULD END HERE")
                chunks.append(text[chunk_start:])
                break
            elif chunk_start + chunk_size <= start_checkpoint[match_item]:

                chunk_end = chunk_start + chunk_size
                print(str(chunk_start)+ " "+str(chunk_end))
                chunks.append(text[chunk_start:chunk_end])
                chunk_start = chunk_end
            elif chunk_start + chunk_size > start_checkpoint[match_item]:
                chunks.append(text[chunk_start:start_checkpoint[match_item]])
                print(str(chunk_start)+ " "+str(start_checkpoint[match_item]))
                chunks.append(text[start_checkpoint[match_item]:end_checkpoint[match_item]])
                print(str(start_checkpoint[match_item])+ " "+str(end_checkpoint[match_item]))
                chunk_start = end_checkpoint[match_item]
                chunk_end = end_checkpoint[match_item]
                match_item += 1
        else:
            if chunk_start + chunk_size >= len(text):
                print(str(chunk_start)+ " "+str(len(text)) )
                print("SHOULD END HERE")
                chunks.append(text[chunk_start:])
                break
            else:
                chunk_end = chunk_start + chunk_size
                print(str(chunk_start)+ " "+str(chunk_end))
                chunks.append(text[chunk_start:chunk_end])
                chunk_start = chunk_end


        # table還好，但我希望image自己一人一個chunk，這樣如果取到相關的image時可以對應到真實相片路徑
    # 合併相鄰的chunks
    merged_chunks = []
    for i in range(len(chunks) - 1):
        if ("end of image" in chunks[i]) and ("end of image" not in chunks[i+1]):
            merged_chunks.append(chunks[i])
            merged_chunks.append(chunks[i+1])
        elif ("end of image" in chunks[i]) and ("end of image" in chunks[i+1]):
            merged_chunks.append(chunks[i])
        else:
            merged_chunks.append(chunks[i] + chunks[i+1])
        if i==(len(chunks)-2) and ("end of image" in chunks[i+1]):
            merged_chunks.append(chunks[i+1])


    now_page = 0
    chunk_page = list()
    print("Number of merged chunks:", len(merged_chunks))
    print("Number of pages in txt_pages_dict:", len(txt_pages_dict))
    for chunk in merged_chunks:
        inner_dict = {"chunk": chunk, "start_page": "", "end_page":""}

        #first_content = 有可能的第一頁的內容
        #last_content = 有可能的最後一頁的內容
        #content = 這之間的內容（可能跨多頁）
        first_content = txt_pages_dict[now_page]["text"]
        content = first_content
        last_content = ""
        forward_step = 0
        page_loc = []
        while now_page < len(txt_pages_dict): #從第一頁到最後一頁
            if (chunk in last_content) and (chunk not in first_content):
                #所有的內容不在原本這頁了，但是在下一頁．這之後就完全忽略前一頁的內容
                now_page += 1
                page_loc.append(now_page +1)
                break
            elif (chunk in content):
                #跨多頁的內容（含第一頁到最後一頁）會在這裡處理
                end_page = now_page + forward_step
                for step in range(now_page, end_page+1):
                    page_loc.append(step+1)
                if last_content != "":
                    now_page = now_page + forward_step - 1
                break  # 找到 chunk 所在的頁面後退出循環
            else:
                #如果在目前的content找不到，就往下一頁找
                forward_step += 1
                #如果已經到最後一頁了，就不再往下找
                if now_page + forward_step >= len(txt_pages_dict):
                    break
                last_content = txt_pages_dict[now_page+forward_step]["text"]
                content += last_content
        print(page_loc)
        if len(page_loc) == 0:
            page_txt = "Not Found"
            inner_dict["start_page"] = page_txt
            inner_dict["end_page"] = page_txt
        else:
            inner_dict["start_page"] = page_loc[0]
            inner_dict["end_page"] = page_loc[-1] #不是-1是因為後面還有一個空格
        chunk_page.append(inner_dict)
        print(str(inner_dict["start_page"]) + " " + str(inner_dict["end_page"]))
    print("===================================")
    return merged_chunks, chunk_page

In [18]:
with open("/content/咒術迴戰2_parse.txt", "r") as file:
    content = file.read()
doc_list = content.split("自由的百科全書")
print(len(doc_list))

17


In [19]:
index = 1
txt_page = []

for output_text in doc_list:
    output_dict = {
        "page": index,
        "text": output_text
    }
    txt_page.append(output_dict)
    index += 1
print(len(txt_page))


17


In [31]:
merge_chunk_content_new(txt_page, chunk_size=500)

(4037, 5207, 'table', '<table>\n    <tr>\n        <th rowspan="2">咒術迴戰</th>\n        <th>Col2</th>\n    </tr>\n    <tr>\n        <td></td>\n    </tr>\n    <tr>\n        <td rowspan="2">台灣中文版漫畫單行本第1卷封面</td>\n        <td></td>\n    </tr>\n    <tr>\n        <td>呪術廻戦 Jujutsu Kaisen</td>\n    </tr>\n    <tr>\n        <td>假名</td>\n        <td>じゅじゅつかいせん</td>\n    </tr>\n    <tr>\n        <td>類型</td>\n        <td>少年漫畫、黑暗奇幻、校園、戰鬥</td>\n    </tr>\n    <tr>\n        <td>正式譯名</td>\n        <td>[註 1] 咒術迴戰 咒术回战</td>\n    </tr>\n    <tr>\n        <td>漫畫</td>\n        <td></td>\n    </tr>\n    <tr>\n        <td>作者</td>\n        <td>芥見下下</td>\n    </tr>\n    <tr>\n        <td>出版社</td>\n        <td>集英社 文化傳信 東立出版社 柒海圖書（代理）、西泠印社出版社（出版）</td>\n    </tr>\n    <tr>\n        <td>連載雜誌</td>\n        <td>週刊少年Jump 寶島少年</td>\n    </tr>\n    <tr>\n        <td>叢書</td>\n        <td>JUMP COMICS CITICOMICS</td>\n    </tr>\n    <tr>\n        <td>連載網站</td>\n        <td>Bilibili</td>\n    </tr>\n    <tr>\n        <td>連載期間<

(['2024/8/27 上午11:47 咒術迴戰 - 維基百科，\n\n# 咒術迴戰\n\n[《咒術迴戰》（日語：呪術廻戦）是日本漫畫家芥見下下的漫畫作品，集英社出版，於漫畫雜誌](https://zh.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E6%BC%AB%E7%95%AB%E5%AE%B6)\n[《週刊少年Jump》2018年第14號開始連載。作者2017年的漫畫作品《東京都立咒術高等專門學校》](https://zh.wikipedia.org/wiki/%E9%80%B1%E5%88%8A%E5%B0%91%E5%B9%B4Jump)\n[是本作的前身也是前傳。截至2023年12月，單行本累計發行量超過9,000萬本（含電子版）。](https://zh.wikipedia.org/wiki/%E5%89%8D%E5%82%B3)\n\n[改編電視動畫第1季於2020年10月至2021年3月期間播映，共24集。改編自前傳的動畫電影《劇場版](https://zh.wikipedia.org/wiki/%E5%92%92%E8%A1%93%E8%BF%B4%E6%88%B0_(%E5%8B%95%E7%95%AB))\n**[咒術迴戰 0》於2021年12月24日上映。電視動畫第2季已於2023年7月至12月期間播映，共23集。](https://zh.wikipedia.org/wiki/%E5%8A%87%E5%A0%B4%E7%89%88_%E5%92%92%E8%A1%93%E8%BF%B4%E6%88%B0_0)**\n\n## 劇情大綱\n\n### 設定\n\n人類的負面情緒累積混雜後會形成咒靈並對人類造成危害，日本每年不明死亡或失蹤年均超過1萬人\n大部分因此造成，只有使用被稱為咒力的能量才得以祓除，而負責祓除的人被稱為「咒術師」。由\n於咒術師是利用負面能量（具體分為咒力和術式）與咒靈戰鬥的人，常常得目睹非術師或同行遇害\n的悲慘死狀，所以必須要有一定程度的瘋狂以及足夠的動機才能擔任此職。平常生活訓練中咒術師\n也必須自主學會控制汲取負面情緒如殺意作為咒力，而且要在激烈波動時也要想辦法不要浪費咒\n力。咒術（術式）跟咒力不同，是與生俱來刻在身體上的能力，因此咒術師的實力大概有八成都是\n依據此天賦，並不

In [16]:
len(content)

53944